In [1]:
import json
import datetime
import time
import requests
import nbimporter
import random
import Proxy_Gatherer as pg
import Init_database as indb
import Insert_py
import Get_py
import psycopg2
from multiprocessing import Process

Importing Jupyter notebook from Proxy_Gatherer.ipynb
Importing Jupyter notebook from Init_database.ipynb
Importing Jupyter notebook from Insert_py.ipynb
Importing Jupyter notebook from Get_py.ipynb


In [3]:
# #--------THIS ONLY RUNS ONE TIME TO CREATE A DATABASE FRAMEWORK--------#
# import Create_Teld_Database as ctd
# db_init = ctd.create_teld_database()

In [2]:
#--------INITIALIZING CONNECTION TO DATABASE--------#
db_name="db_teld"
user_name="postgres"
password = "postgres"
host="127.0.0.1"
port="5432"
conn = indb.init_db(db_name, user_name, password, host, port)

In [3]:
##------- Specify request header ------- ##
hdr = {
    'accept': "*/*",
    'user-agent': "Teld/3.4.0 (iPhone; iOS 10.3.3; Scale/2.00)",
    'cookie': "TELDAppID=",
    'content-length': "111",
    'connection': "keep-alive",
    'accept-encoding': "gzip, deflate",
    'device': "app_version=3.4.0&os_version=10.3.3&client=ios&device_name=iPhone 6s (A1633/A1688/A1691/A1700)&device_id=38CD294C-CC36-4998-9074-C7F798EE6087&city_code=4403&city_name=%E6%B7%B1%E5%9C%B3&lat=0.000000&lng=0.000000&network=wifi&location_city_name=",
    'host': "basesg.teld.cn",
    'content-type': "application/x-www-form-urlencoded",
    'cache-control': "no-cache",
    'postman-token': "4dd0a429-35ad-1863-c8f5-18e0f7805f60"
    }

In [6]:
#------- Gathering proxies for visiting teld.cn -------##
# gatherproxy = pg.ProxyGatherer(hdr)
# proxies = gatherproxy.getproxies()
# # t = time.time()
# proxies_pool = gatherproxy.proxies_cleanup(proxies) # Could also specify timeout limit you prefer, 
#                                                                 #format--(connect time, read time)
# print(time.time()-t)  ## usually this checking process takes a minute or so and usable proxies are around 80+.
# proxies_pool=[{'https': '221.7.153.122:8118'},
#  {'http': '175.42.102.119:8118'},
#  {'https': '218.15.25.153:808'},
#  {'http': '106.44.81.222:8118'},
#  {'https': '113.128.91.126:48888'},
#  {'https': '39.82.38.99:8118'},
#  {'http': '117.60.208.87:8118'},
#  {'http': '113.128.90.45:48888'},
#  {'http': '118.187.58.34:53281'},
#  {'http': '49.70.146.211:8118'},
#  {'http': '113.128.91.101:48888'},
#  {'http': '61.135.217.7:80'},
#  {'http': '222.141.12.83:8118'},
#  {'https': '123.131.28.9:53281'},
#  {'http': '222.140.5.63:9999'},
#  {'http': '218.6.145.11:9797'},
#  {'http': '218.28.58.150:53281'},
#  {'https': '61.160.208.222:8080'},
#  {'http': '111.194.152.246:9000'},
#  {'https': '42.202.130.246:3128'},
#  {'http': '113.200.159.155:9999'},
#  {'http': '183.15.80.84:9797'},
#  {'http': '116.228.221.54:80'},
#  {'https': '119.90.63.3:3128'},
#  {'https': '222.196.33.254:3128'},
#  {'http': '125.46.0.62:53281'},
#  {'http': '180.168.179.193:8080'},
#  {'https': '36.249.173.63:8118'},
#  {'https': '111.192.49.21:9000'},
#  {'https': '113.128.90.76:48888'},
#  {'https': '27.203.131.118:53281'},
#  {'https': '183.66.64.120:3128'},
#  {'https': '123.119.74.75:8118'},
#  {'https': '144.255.229.192:8118'},
#  {'https': '120.76.55.49:8088'},
#  {'https': '116.196.123.38:3128'},
#  {'https': '220.248.207.105:53281'},
#  {'https': '220.166.64.174:3128'},
#  {'https': '115.159.252.79:8080'},
#  {'http': '113.128.90.168:48888'},
#  {'http': '182.96.194.170:8118'},
#  {'http': '49.77.61.124:8118'},
#  {'http': '27.46.74.43:9999'},
#  {'http': '113.128.91.13:48888'},
#  {'http': '113.128.90.97:48888'},
#  {'http': '116.23.136.21:9999'},
#  {'http': '111.155.124.81:8123'},
#  {'http': '220.165.174.9:8118'},
#  {'http': '211.94.69.74:8080'},
#  {'http': '112.226.233.252:8118'},
#  {'http': '144.52.191.109:53281'},
#  {'http': '222.141.12.83:8118'},
#  {'http': '1.119.193.36:8080'},
#  {'http': '125.45.87.12:9999'},
#  {'http': '139.129.166.68:3128'},
#  {'http': '180.168.179.193:8080'},
#  {'http': '125.46.0.62:53281'},
#  {'http': '61.135.217.7:80'}]

In [7]:
# F = open('Error_log.txt','w') 

In [4]:
def pillState_scrap(conn, hdr, pillcode, error_log=None, proxies_pool=None): 
    """
        hdr - dictionary, detailed spec for headers
        pillCode - string, each charging pile has its own pile code
        error_log - specify file to write errors
        proxies_pool - contains all the proxies address

    """
    F = error_log
    #If encounter error, try one more request with a different proxy
    try:
        pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=10000)
    except ValueError as e:
        if F is not None:
            F.write(pile[0])
            F.write(str(e))
            F.write('\n')
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=10000)
        except ValueError as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
    except (requests.exceptions.ChunkedEncodingError) as e:
        if F is not None:
            F.write(pile[0])
            F.write(str(e))
            F.write('\n')
        try:
            pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=10000)
        except ValueError as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
        except (requests.exceptions.ChunkedEncodingError) as e:
            if F is not None:
                F.write(pile[0])
                F.write(str(e))
                F.write('\n')
            pill_detailedInfo = None
    if pill_detailedInfo is not None:
        pillCode = str(pill_detailedInfo['pillCode'])
        stateCode = str(pill_detailedInfo['stateCode'])
        stateCodeV3 = str(pill_detailedInfo['stateCodeV3'])
        if pill_detailedInfo['stateName'] is not None:
            stateName = str(pill_detailedInfo['stateName'].encode('utf-8'))
        else:
            stateName = ''
        if pill_detailedInfo['stateNameV3'] is not None:
            stateNameV3 = str(pill_detailedInfo['stateNameV3'].encode('utf-8'))
        else:
            stateNameV3 = ''
        conn = Insert_py.insert_state_info(conn, pillCode, stateCode, stateName, stateCodeV3, stateNameV3)

In [5]:
# ##------- Every once a while runs it update station and pile fixed info -------##
# #--------------------------------------------#
# #------- Gather all stations  -------# ****NEED TO BE FIXED 09.17.2017
# station_info = Get_py.get_allStations(hdr)
# print(len(station_info))
# for station in station_info:
#     stationId = str(station['id'])  
# #     print(stationId)
#     #------- Gather individual stations details -------#
#     station_detailedInfo = Get_py.get_oneStation_APP(hdr, stationId)
#     conn = Insert_py.insert_singStation_details(conn, station_detailedInfo)
#     #------- Gather individual pile code -------#
#     piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
#     conn = Insert_py.insert_pilecode(conn, stationId, piles_info)
# #------- Gather inidividual pile details -------#
# #------- Request pilecode from db for states -------#
try:
    cur = conn.cursor()
    cur.execute("select pilecode from tbl_APP_pilecode;")
    pillcode_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(pillcode_list))

# # ------- Requests stationID from db for user charging activities -------#
try:
    cur = conn.cursor()
    cur.execute("select stationid from tbl_app_eachstations;")
    station_list = cur.fetchall()
    cur.close()
except psycopg2.Error as e:
    print(e)
print(len(station_list))

# reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15)
# i = 0
# for pile in pilecode_list:
#     #If encounter error, try one more request with a different proxy
#     try:
#         pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#     except ValueError as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     except (requests.exceptions.ChunkedEncodingError) as e:
#         try:
#             pile_detailedInfo, proxies_pool = Get_py.get_onePile_APP(F, hdr, pile[0], proxies_pool, time_out=1000)
#         except ValueError as e:
#             pile_detailedInfo = None
#         except (requests.exceptions.ChunkedEncodingError) as e:
#             pile_detailedInfo = None
#     if pile_detailedInfo is not None:
#         conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
#     else:
#         print("Running None Func.")
#         conn = Insert_py.insert_singPile_none(conn, pile_detailedInfo, pile[0])
#     i += 1
#     if datetime.datetime.now() >= reportTime:
#         reportTime = datetime.datetime.now() + datetime.timedelta(minutes=15) 
#         print("Successfully run {} piles".format(i))
# #211 Running None Func.
# #71585 piles.

71585
8866


In [27]:
stationId = "d4035b02-5f27-4ae9-b0b1-dd10010f5b7e"

url = "https://baseapi.teld.cn/api/GetChargeListBySta/StaV3"

querystring = {"itemNumPerPage":"40","pageNum":"1","staID":"d4035b02-5f27-4ae9-b0b1-dd10010f5b7e"}


headers = {
    'cache-control': "no-cache",
    'postman-token': "928814f4-60c2-7c0c-a892-12409b8990bd"
    }

# response = requests.request("GET", url, headers=headers, params=querystring)
chargeList_detailedInfo = Get_py.get_chargelist_APP(stationId)
chargeList_detailedInfo

[{u'PileName': u'104\u53f7',
  u'carTypeName': u'\u672a\u77e5\u8f66\u578b',
  u'chargeTime': u'6\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'\u81f3***\u751f'},
 {u'PileName': u'102\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'7\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'105\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'105\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'104\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNickName': u'1***2'},
 {u'PileName': u'101\u53f7',
  u'carTypeName': u'\u745e\u9a70EC35',
  u'chargeTime': u'8\u5c0f\u65f6\u524d',
  u'hasCertificate': False,
  u'userNi

In [7]:
i = 0
fail = 0
hour_uni = u'\u5c0f\u65f6\u524d'
minute_uni = u'\u5206\u949f\u524d'
for stationID in station_list[100:]:
    stationID = stationID[0]
    try:
        chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID)
    except requests.exceptions.ConnectionError as e:
        try:
            chargeList_detailedInfo = Get_py.get_chargelist_APP(stationID)
        except requests.exceptions.ConnectionError as e:
            fail += 1
            pass
    if len(chargeList_detailedInfo) == 0:
        i += 1
    else:
        for user_act in chargeList_detailedInfo:
            pileName = str(user_act['PileName'].encode('utf-8'))
            if user_act['userNickName'] is not None:
                userNickName = str(user_act['userNickName'].encode('utf-8'))
            else:
                userNickName = None
            if user_act['carTypeName'] is not None:
                carTypeName = str(user_act['carTypeName'].encode('utf-8'))
            else:
                carTypeName = None
            startTime = user_act['chargeTime']
            #AI detect startTime
            if len(startTime) < 6:
            #if in the expression of hours:
                if hour_uni in startTime:
                    idx_hour = len(startTime)-3
                    hours = int(startTime[:idx_hour])
                    startTime = startTime = datetime.datetime.now() - datetime.timedelta(hours=hours)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
                    startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
            #if in the expression of minutes:
                if minute_uni in startTime:
                    idx_min = len(startTime)-3
                    minutes = int(startTime[:idx_min])
                    startTime = startTime = datetime.datetime.now() - datetime.timedelta(minutes=minutes)
#                     startTime = '{:%Y-%m-%d %H:%M:%S}'.format(startTime)+"-07"
                    startTime = '{:%Y-%m-%d %H:%M}'.format(startTime)+"-07"
            elif len(startTime) >= 6:
                startTime = '2017-' + startTime + '-07'                    
            conn = Insert_py.insert_userActivity_info(conn, startTime, pileName, stationID, carTypeName, userNickName)
print(str(i) + " stations do not have activity history.")
print(str(fail) + " connections failed attempt.")

invalid input syntax for type timestamp with time zone: "刚刚"
LINE 4: ... (SELECT 1 FROM tbl_app_useractivity where startTime='刚刚')
                                                                ^

4230 stations do not have activity history.
3 connections failed attempt.


('memory use:', 134.76171875, 'MB')


In [10]:
## ----- Test for single looping of multiprocessing post_request and insert -----##
import os
import psutil
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('Before runnning script, memory use:', memoryUse, 'MB')
# %load_ext memory_profiler
# from memory_profiler import profile
# @profile
# def test():
loop_num = 4
for run in range(loop_num):
    tmp_pilllist = pillcode_list[run*900:(run+1)*900]
    t = time.time()
    if __name__ == '__main__':
        proc_record = []
        conn_list = []
        for i in range(len(tmp_pilllist)):
            conn = indb.init_db(db_name, user_name, password, host, port)
            conn_list.append(conn)
            p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0],))
            p.start()
            proc_record.append(p)
        for p in proc_record:
            p.join()
        for conn in conn_list:
            conn.close()
        for p in proc_record:
            p.terminate()
        del proc_record
        del conn_list
        print("success, time costs is " + str(time.time()-t))   
pid = os.getpid()
py = psutil.Process(pid)
memoryUse = py.memory_info()[0]/2.**20 #in MB
print('After runnning script,memory use:', memoryUse, 'MB')

('Before runnning script, memory use:', 150.51171875, 'MB')


OperationalError: could not create socket: Too many open files


In [15]:
%memit test

/usr/local/lib/python2.7/dist-packages/memory_profiler.py:163: UserWarning: psutil module not found. memory_profiler will be slow
  warnings.warn("psutil module not found. memory_profiler will be slow")


peak memory: 132.62 MiB, increment: 0.26 MiB


In [6]:
def main():
    """The main process"""
    global displaytime
    global endTime 
    global max_proc
    global loop_no 
    global rest_no
    proxies_pool = None
    t = time.time()
    while True:
        if datetime.datetime.now() >= endTime:
            break
        for loopNo in range(loop_no):
            tmp_pilllist = pillcode_list[loopNo*max_proc:(loopNo+1)*max_proc]
            proc_record = []
            conn_list = []                
            for i in range(len(tmp_pilllist)):
                conn = indb.init_db(db_name, user_name, password, host, port)
                conn_list.append(conn)
                p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0],))
                p.start()
                proc_record.append(p)
            for p in proc_record:
                p.join()
            for conn in conn_list:
                conn.close()
            for p in proc_record:
                p.terminate()
            del proc_record
            del conn_list
            if datetime.datetime.now() >= displaytime:
                print("successfully looped {} times".format(loopNo+1))
                displaytime += datetime.timedelta(hours=1)
        tmp_pilllist = pillcode_list[loop_no*max_proc:]
        proc_record = []
        conn_list = [] 
        for i in range(len(tmp_pilllist)):
            conn = indb.init_db(db_name, user_name, password, host, port)
            conn_list.append(conn)
            p = Process(target=pillState_scrap, args=(conn, hdr, tmp_pilllist[i][0],))
            p.start()
            proc_record.append(p)
        for p in proc_record:
            p.join()
        for conn in conn_list:
            conn.close()
        for p in proc_record:
            p.terminate()
        del proc_record
        del conn_list
        print("time for a sleep zz..")
        print("One process runs for "+str(time.time()-t))
        time.sleep(900)

if __name__ == '__main__':
    displaytime = datetime.datetime.now() + datetime.timedelta(hours=1)
    endTime = datetime.datetime.now() + datetime.timedelta(hours=12)
    max_proc = 900
    loop_no = len(pillcode_list)//max_proc
    rest_no = len(pillcode_list)%max_proc
    main()

successfully looped 48 times
failed
failed
failed
failed
time for a sleep zz..
One process runs for 6504.07678986
successfully looped 1 times
failed


Process Process-80386:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-7b6fbd3a4e95>", line 12, in pillState_scrap
    pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=10000)
  File "Get_py.ipynb", line 24, in get_onePill_APP
    "def get_station(header, start_page=1, end_page=2, row_nmb=10):\n",
  File "/usr/local/lib/python2.7/dist-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 508, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 618, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/l

successfully looped 10 times
successfully looped 11 times
successfully looped 12 times
failed
failed
successfully looped 49 times
time for a sleep zz..
One process runs for 24116.0401859
successfully looped 3 times
successfully looped 47 times
failed
time for a sleep zz..
One process runs for 31603.541971
failed
failed
failed
failed
successfully looped 1 times
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
successfully looped 36 times
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed


Process Process-281770:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-7b6fbd3a4e95>", line 12, in pillState_scrap
    pill_detailedInfo, proxies_pool = Get_py.get_onePill_APP(hdr, pillcode, F, proxies_pool, time_out=10000)
  File "Get_py.ipynb", line 24, in get_onePill_APP
    "def get_station(header, start_page=1, end_page=2, row_nmb=10):\n",
  File "/usr/local/lib/python2.7/dist-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 508, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 618, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/

successfully looped 75 times
successfully looped 76 times
successfully looped 77 times
time for a sleep zz..
One process runs for 49722.2225559


In [ ]:
len(proxies_pool)
# F.close()

In [ ]:
proxies_pool

In [ ]:
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pile[0]
# proxy_idx = random.randint(0,len(proxies_pool)-1)
# pile_resp = requests.get(REQUEST_URL, proxies=proxies_pool[proxy_idx], headers=hdr, timeout=1000)

In [ ]:
# pilecode = '1101060314110'
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, pilecode)
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)
# REQUEST_URL = 'http://baseapi.teld.cn/api/GetPillDetails/StaV2?pillCode='+pilecode
# pile_resp = requests.get(REQUEST_URL, headers=hdr)


In [ ]:
# json_requestData = pile_resp.json()
# json_requestData
# pile_detailedInfo = json_requestData['data']['terminal']

In [ ]:
# stationId = '044a0e7c-419e-477e-aafb-8d4b9eea1475'
# piles_info = Get_py.get_piles(stationId, proxies_pool, hdr)
# pile_detailedInfo = Get_py.get_onePile_APP(hdr, piles_info[3]['code'])

In [ ]:
# pile_detailedInfo
# conn = Insert_py.insert_singPile_details(conn, pile_detailedInfo)

In [ ]:
##------- Try fetching all the data through psycopg2 -------## 
# try:
#     cur = conn.cursor()
#     cur.execute("select stationid from tbl_app_eachstations limit 10;")
#     exists = cur.fetchall()
#     cur.close()
# except psycopg2.Error as e:
#     print(e)
# for i in range(len(exists)):
#     print(exists[i][0])

In [ ]:
# REQUEST_URL = 'https://www.teld.cn/StationNetwork/GetStationNetword?ProvinceName=&CityName=&KeyWords=&RegionName=& \
#                             type=&page=1&rows=10'
# # req = urllib.request.Request(REQUEST_URL) 
# hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/ \
#                        59.0.3071.115 Safari/537.36', 
#        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8'}
# response = urllib2.Request(REQUEST_URL, headers=hdr)

# try:
#     page = urllib2.urlopen(response)
# except urllib2.HTTPError, e:
#     print e.fp.read()
    
# content = json.load(page)
# print(content['rows'][0])
# response_data = urllib.request.urlopen(REQUEST_URL)
# d = json.load(response_data)
# print('number of results %d' % (len(d)))
# type(response_data)